In [23]:
import pandas as pd
import numpy as np
import copy
import os
from tqdm import tqdm

In [24]:
import matplotlib.pyplot as plt

## For case studies

In [25]:
benchmark_directory = "/projects/mcmm/results_final/"

In [26]:
def get_info_from_filename(np_fname):
    splitted = np_fname.lower().split("_")
    func = splitted[0]
    dim = int(splitted[1])
    
    algorithm = None
    if "basinhopping" in np_fname:
        assert algorithm is None; algorithm = "basinhopping"
    elif "differential_evolution" in np_fname:
        assert algorithm is None; algorithm = "differential_evolution"
    elif "direct" in np_fname:
        assert algorithm is None; algorithm = "direct"
    elif "dual_annealing" in np_fname:
        assert algorithm is None; algorithm = "dual_annealing"
    elif "shgo" in np_fname:
        assert algorithm is None; algorithm = "shgo"
    elif "simulated_annealing" in np_fname:
        assert algorithm is None; algorithm = "simulated_annealing"
    elif "mcir" in np_fname:
        assert algorithm is None; algorithm = "mcir"
    
    assert algorithm is not None, "algorithm not found in {}".format(np_fname)
    
    info = {
        "function": func,
        "dimension": dim,
        "algorithm": algorithm,
    }
    return info

In [27]:
all_results = []

for filename in os.listdir(benchmark_directory):
    assert filename.endswith("_Y.npy")
    result_dict = get_info_from_filename(filename)
    result_dict['obj_list'] = np.load(os.path.join(benchmark_directory, filename))
    all_results.append(result_dict)

In [28]:
# Generate dataframe for results

In [29]:
def process_list(nums):
    result = []
    min_num = float('inf')  # initialize minimum value to positive infinity
    for num in nums:
        if num < min_num:
            min_num = num
        result.append(min_num)
    return result

In [30]:
result_df = pd.DataFrame(all_results)
result_df['min_found_list'] = result_df['obj_list'].apply(process_list)
result_df = result_df.sort_values(["function", "dimension", "algorithm"])
result_df.to_pickle("raw_result_df.pkl")
result_df

,function,dimension,algorithm,obj_list,min_found_list
59,ackley,10,basinhopping,"[21.23412, 21.23412, 21.23412, 21.23412, 21.23...","[21.23412, 21.23412, 21.23412, 21.23412, 21.23..."
105,ackley,10,basinhopping,"[21.551155, 21.551155, 21.551155, 21.551155, 2...","[21.551155, 21.551155, 21.551155, 21.551155, 2..."
339,ackley,10,basinhopping,"[21.037682, 21.037682, 21.037682, 21.037682, 2...","[21.037682, 21.037682, 21.037682, 21.037682, 2..."
351,ackley,10,basinhopping,"[20.554607, 20.554607, 20.554607, 20.554607, 2...","[20.554607, 20.554607, 20.554607, 20.554607, 2..."
436,ackley,10,basinhopping,"[20.38853, 20.38853, 20.38853, 20.38853, 20.38...","[20.38853, 20.38853, 20.38853, 20.38853, 20.38..."
...,...,...,...,...,...
54,watson,32,shgo,[],[]
148,watson,32,shgo,[],[]
222,watson,32,shgo,[],[]
288,watson,32,shgo,[],[]


In [35]:
# Generate statistics for results

In [36]:
stat_df = result_df.groupby(['function', 'dimension', 'algorithm']).agg(
    obj_lists=('obj_list', list),
    min_found_lists=('min_found_list', list)
).reset_index()
stat_df

,function,dimension,algorithm,obj_lists,min_found_lists
0,ackley,10,basinhopping,"[[21.23412, 21.23412, 21.23412, 21.23412, 21.2...","[[21.23412, 21.23412, 21.23412, 21.23412, 21.2..."
1,ackley,10,differential_evolution,"[[20.970161, 21.383083, 21.541191, 21.367462, ...","[[20.970161, 20.970161, 20.970161, 20.970161, ..."
2,ackley,10,direct,"[[4.552539, 17.018728, 17.45362, 17.018728, 17...","[[4.552539, 4.552539, 4.552539, 4.552539, 4.55..."
3,ackley,10,dual_annealing,"[[21.551155, 21.254986, 21.6148, 20.65361, 21....","[[21.551155, 21.254986, 21.254986, 20.65361, 2..."
4,ackley,10,mcir,"[[21.122943878173828, 21.122943878173828, 21.1...","[[21.122943878173828, 21.122943878173828, 21.1..."
...,...,...,...,...,...
117,watson,32,differential_evolution,"[[307303.9192011102, 300601.11886811716, 20247...","[[307303.9192011102, 300601.11886811716, 30060..."
118,watson,32,direct,"[[528043.0017411856, 528043.0017411856, 528043...","[[528043.0017411856, 528043.0017411856, 528043..."
119,watson,32,dual_annealing,"[[452408.46655145474, 37596435.013013594, 6810...","[[452408.46655145474, 452408.46655145474, 6810..."
120,watson,32,mcir,"[[651574.6720559866, 651574.6720559866, 651574...","[[651574.6720559866, 651574.6720559866, 651574..."


In [37]:
def calculate_mean_and_std_of_lists(lists):
    pd_lists = pd.DataFrame(lists)
    means = pd_lists.mean(axis = 0).to_numpy()
    std = pd_lists.std(axis=0).fillna(0).to_numpy()
    
    return {
        "avg_min_found": means,
        "std_min_found": std
    }

In [38]:
applied_df = stat_df.apply(
    lambda row: calculate_mean_and_std_of_lists(row.min_found_lists), 
    axis='columns', 
    result_type='expand'
)
stat_df = pd.concat([stat_df, applied_df], axis='columns')

In [39]:
stat_df.to_pickle("result_df.pkl")
stat_df

,function,dimension,algorithm,obj_lists,min_found_lists,avg_min_found,std_min_found
0,ackley,10,basinhopping,"[[21.23412, 21.23412, 21.23412, 21.23412, 21.2...","[[21.23412, 21.23412, 21.23412, 21.23412, 21.2...","[20.95322036743164, 20.95322036743164, 20.9532...","[0.4799298346042633, 0.4799298346042633, 0.479..."
1,ackley,10,differential_evolution,"[[20.970161, 21.383083, 21.541191, 21.367462, ...","[[20.970161, 20.970161, 20.970161, 20.970161, ...","[21.358928680419922, 21.04205322265625, 20.861...","[0.23468589782714844, 0.45797276496887207, 0.3..."
2,ackley,10,direct,"[[4.552539, 17.018728, 17.45362, 17.018728, 17...","[[4.552539, 4.552539, 4.552539, 4.552539, 4.55...","[4.552539, 4.552539, 4.552539, 4.552539, 4.552...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,ackley,10,dual_annealing,"[[21.551155, 21.254986, 21.6148, 20.65361, 21....","[[21.551155, 21.254986, 21.254986, 20.65361, 2...","[20.95322036743164, 20.893985748291016, 20.893...","[0.4799298346042633, 0.39916253089904785, 0.39..."
4,ackley,10,mcir,"[[21.122943878173828, 21.122943878173828, 21.1...","[[21.122943878173828, 21.122943878173828, 21.1...","[20.95499038696289, 20.95499038696289, 20.9549...","[0.4690623126870676, 0.4690623126870676, 0.469..."
...,...,...,...,...,...,...,...
117,watson,32,differential_evolution,"[[307303.9192011102, 300601.11886811716, 20247...","[[307303.9192011102, 300601.11886811716, 30060...","[6844500.333347577, 1662800.6946316964, 136326...","[11484157.537096405, 1726414.08927726, 1784276..."
118,watson,32,direct,"[[528043.0017411856, 528043.0017411856, 528043...","[[528043.0017411856, 528043.0017411856, 528043...","[528043.0017411856, 528043.0017411856, 528043....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
119,watson,32,dual_annealing,"[[452408.46655145474, 37596435.013013594, 6810...","[[452408.46655145474, 452408.46655145474, 6810...","[37751606.73584604, 3791722.58588769, 2881944....","[62101331.395239055, 5219180.813468672, 538362..."
120,watson,32,mcir,"[[651574.6720559866, 651574.6720559866, 651574...","[[651574.6720559866, 651574.6720559866, 651574...","[588816.5247719139, 588816.5247719139, 588816....","[88753.42303854393, 88753.42303854393, 88753.4..."


## Ablation studies

In [74]:
def get_info_from_ablation_filename(np_fname):
    splitted = np_fname.lower().split("_")
    func = splitted[0]
    dim = int(splitted[1])
    study_type = splitted[2]
    
    info = {
        "function": func,
        "dimension": dim,
        "study": study_type,
    }
    return info

In [75]:
ablation_directory = "/projects/mcmm/ablation_final/"

In [76]:
all_results = []
for filename in os.listdir(ablation_directory):
    assert filename.endswith("_Y.npy")
    result_dict = get_info_from_ablation_filename(filename)
    result_dict['obj_list'] = np.load(os.path.join(ablation_directory, filename))
    all_results.append(result_dict)

In [77]:
result_df = pd.DataFrame(all_results)
result_df['min_found_list'] = result_df['obj_list'].apply(process_list)
result_df = result_df.sort_values(["function", "dimension", "study"])
result_df.to_pickle("ablation_raw_result_df.pkl")
result_df

,function,dimension,study,obj_list,min_found_list
18,ackley,50,boxhigh,"[21.253238677978516, 21.253238677978516, 21.25...","[21.253238677978516, 21.253238677978516, 21.25..."
19,ackley,50,boxhigh,"[21.12533187866211, 21.12533187866211, 21.1253...","[21.12533187866211, 21.12533187866211, 21.1253..."
21,ackley,50,boxhigh,"[21.27104949951172, 21.27104949951172, 21.2710...","[21.27104949951172, 21.27104949951172, 21.2710..."
24,ackley,50,boxhigh,"[21.203022003173828, 21.203022003173828, 21.20...","[21.203022003173828, 21.203022003173828, 21.20..."
51,ackley,50,boxhigh,"[21.243709564208984, 21.243709564208984, 21.24...","[21.243709564208984, 21.243709564208984, 21.24..."
...,...,...,...,...,...
73,michalewicz,50,original,"[-4.662331581115723, -4.662331581115723, -4.66...","[-4.662331581115723, -4.662331581115723, -4.66..."
78,michalewicz,50,original,"[-5.721030235290527, -5.721030235290527, -5.72...","[-5.721030235290527, -5.721030235290527, -5.72..."
42,watson,32,local-opt,"[200290.4072673328, 76608.51186238528, 3356366...","[200290.4072673328, 76608.51186238528, 76608.5..."
45,watson,32,original,"[526058.3774878411, 526058.3774878411, 526058....","[526058.3774878411, 526058.3774878411, 526058...."


In [78]:
stat_df = result_df.groupby(['function', 'dimension', 'study']).agg(
    obj_lists=('obj_list', list),
    min_found_lists=('min_found_list', list)
).reset_index()
stat_df

,function,dimension,study,obj_lists,min_found_lists
0,ackley,50,boxhigh,"[[21.253238677978516, 21.253238677978516, 21.2...","[[21.253238677978516, 21.253238677978516, 21.2..."
1,ackley,50,boxlow,"[[21.340312957763672, 21.340312957763672, 21.3...","[[21.340312957763672, 21.340312957763672, 21.3..."
2,ackley,50,exphigh,"[[21.444210052490234, 21.444210052490234, 21.4...","[[21.444210052490234, 21.444210052490234, 21.4..."
3,ackley,50,explow,"[[21.39307403564453, 21.39307403564453, 21.393...","[[21.39307403564453, 21.39307403564453, 21.393..."
4,ackley,50,lbhigh,"[[21.319332122802734, 21.319332122802734, 21.3...","[[21.319332122802734, 21.319332122802734, 21.3..."
5,ackley,50,lblow,"[[21.17459487915039, 21.17459487915039, 21.174...","[[21.17459487915039, 21.17459487915039, 21.174..."
6,ackley,50,local-opt,"[[21.515838623046875, 21.25113296508789, 21.25...","[[21.515838623046875, 21.25113296508789, 21.25..."
7,ackley,50,no-learning,"[[21.0770263671875, 21.0770263671875, 21.07702...","[[21.0770263671875, 21.0770263671875, 21.07702..."
8,ackley,50,no-local-opt,"[[21.523216247558594, 21.523216247558594, 21.3...","[[21.523216247558594, 21.523216247558594, 21.3..."
9,ackley,50,nodeexphigh,"[[21.21166229248047, 21.21166229248047, 21.211...","[[21.21166229248047, 21.21166229248047, 21.211..."


In [79]:
applied_df = stat_df.apply(
    lambda row: calculate_mean_and_std_of_lists(row.min_found_lists), 
    axis='columns', 
    result_type='expand'
)
stat_df = pd.concat([stat_df, applied_df], axis='columns')

In [80]:
stat_df.to_pickle("ablation_result_df.pkl")
stat_df

,function,dimension,study,obj_lists,min_found_lists,avg_min_found,std_min_found
0,ackley,50,boxhigh,"[[21.253238677978516, 21.253238677978516, 21.2...","[[21.253238677978516, 21.253238677978516, 21.2...","[21.219270324707033, 21.219270324707033, 21.21...","[0.05813916062527396, 0.05813916062527396, 0.0..."
1,ackley,50,boxlow,"[[21.340312957763672, 21.340312957763672, 21.3...","[[21.340312957763672, 21.340312957763672, 21.3...","[21.35849609375, 21.35849609375, 21.3584960937...","[0.13442748777655683, 0.13442748777655683, 0.1..."
2,ackley,50,exphigh,"[[21.444210052490234, 21.444210052490234, 21.4...","[[21.444210052490234, 21.444210052490234, 21.4...","[21.25697479248047, 21.25697479248047, 21.2569...","[0.21413986810752586, 0.21413986810752586, 0.2..."
3,ackley,50,explow,"[[21.39307403564453, 21.39307403564453, 21.393...","[[21.39307403564453, 21.39307403564453, 21.393...","[21.277609252929686, 21.277609252929686, 21.27...","[0.1986563006285194, 0.1986563006285194, 0.198..."
4,ackley,50,lbhigh,"[[21.319332122802734, 21.319332122802734, 21.3...","[[21.319332122802734, 21.319332122802734, 21.3...","[21.202345275878905, 21.202345275878905, 21.20...","[0.19453338790692074, 0.19453338790692074, 0.1..."
5,ackley,50,lblow,"[[21.17459487915039, 21.17459487915039, 21.174...","[[21.17459487915039, 21.17459487915039, 21.174...","[21.208272552490236, 21.208272552490236, 21.20...","[0.15277253720062095, 0.15277253720062095, 0.1..."
6,ackley,50,local-opt,"[[21.515838623046875, 21.25113296508789, 21.25...","[[21.515838623046875, 21.25113296508789, 21.25...","[21.28687629699707, 21.20069999694824, 21.2000...","[0.16089645616233283, 0.15890722237908333, 0.1..."
7,ackley,50,no-learning,"[[21.0770263671875, 21.0770263671875, 21.07702...","[[21.0770263671875, 21.0770263671875, 21.07702...","[21.272610473632813, 21.272610473632813, 21.27...","[0.14408538028808102, 0.14408538028808102, 0.1..."
8,ackley,50,no-local-opt,"[[21.523216247558594, 21.523216247558594, 21.3...","[[21.523216247558594, 21.523216247558594, 21.3...","[21.316805267333983, 21.316805267333983, 21.14...","[0.12612812069851692, 0.12612812069851692, 0.2..."
9,ackley,50,nodeexphigh,"[[21.21166229248047, 21.21166229248047, 21.211...","[[21.21166229248047, 21.21166229248047, 21.211...","[21.114319610595704, 21.114319610595704, 21.11...","[0.08534272563178558, 0.08534272563178558, 0.0..."
